In [ ]:
%pylab inline

In [4]:
import pandas as pd
import scipy.stats as stats

In [111]:
!ls ../data

glove.6B.50d.txt	    glove.twitter.27B.25d.txt  test.csv
glove.twitter.27B.100d.txt  glove.twitter.27B.50d.txt  train.csv
glove.twitter.27B.200d.txt  sample_submission.csv


## Preliminary Data Discovery

Data seem to be quite simple: just `id`, text itself and one of six toxicity indicators.

**Problem:** multilabel binary classification.

**Data**: unprocessed comments. This data cannot be taken for granted and should be inspected / munged.

In [ ]:
!head ../data/train.csv

### Target Discovery and Analysis

In [ ]:
# TODO

In [5]:
import telegram
def send_to_telegram(msg):

    """Send appropriate links to telegram channel"""

    bot = telegram.Bot(token='523412387:AAHhEckKtZiCoSG6Pd3ZGtp4-JbL06I8H2E')
    chat_id = -1001371737931
    bot.send_message(chat_id=chat_id, text=msg)
    print(msg)

In [ ]:
send_to_telegram('Эй слыш ну чо')

In [ ]:
1

## Sample LSTM Training

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [15]:
from keras.callbacks import Callback

In [112]:
path = '../data/'
EMBEDDING_FILE=f'{path}glove.840B.300d.txt'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [113]:
embed_size = 300 # how big is each word vector
max_features = 15000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a comment to use

In [114]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [115]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [120]:
def get_coefs(word, *arr):
    try:
        return word, np.asarray(arr, dtype='float32')
    except:
        return
    
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding='utf-8')) if 

TypeError: cannot convert dictionary update sequence element #52343 to a sequence

In [ ]:
from itertools import chain


In [13]:
send_to_telegram('Embeddings calc started.')
all_embs = np.stack(chain.from_iterable(embeddings_index.values()))
# all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
print(f'Embeddings mean: {emb_mean}, embeddings std: {emb_std}')
send_to_telegram('Embeddings calc started.')

Embeddings mean: 0.00906506460160017, embeddings std: 0.44281086325645447


In [14]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [106]:
class TelegramLogger(Callback):
    def __init__(self, msg):
        self.msg = msg
        super()
        
    def on_train_begin(self, logs={}):
        send_to_telegram(self.msg)
        
    def on_epoch_begin(self, epoch, logs={}):
        super()
        
    def on_train_end(self, logs={}):
        send_to_telegram('Completed.')
    
    def on_epoch_end(self, epoch, logs={}):
        send_to_telegram('Epoch #{}, train_acc: {:.5f}'.format(epoch + 1, logs.get('acc')))
 
    def on_batch_begin(self, batch, logs={}):
        super()
 
    def on_batch_end(self, batch, logs={}):
        if logs.get('batch') and logs['batch'] % 500 == 0:
            send_to_telegram('Batch #{}, train_acc: {:.5f}'.format(logs.get('batch'), logs.get('acc')))

In [108]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [109]:
s = []
def print_summary(line):
    global s
    s.append(line[:65])

model.summary(line_length=100, print_fn=print_summary)

def cleanup_summary(summary):
    s = ["CURRENTLY TRAINING:"]
    for line in summary[1:-4]:
        if line.startswith('___'): continue
        else:
            s.append(line)
            
    s.append('BATCH SIZE 32, EPOCHS 2')
    return s

intro_msg = '\n'.join(cleanup_summary(s))

tlg = TelegramLogger(intro_msg)
model.fit(X_t, y, batch_size=32, epochs=2, callbacks=[tlg]);

CURRENTLY TRAINING:
Layer (type)                                 Output Shape        
input_14 (InputLayer)                        (None, 120)         
embedding_14 (Embedding)                     (None, 120, 200)    
bidirectional_14 (Bidirectional)             (None, 120, 100)    
global_max_pooling1d_14 (GlobalMaxPooling1D) (None, 100)         
dense_27 (Dense)                             (None, 50)          
dropout_14 (Dropout)                         (None, 50)          
dense_28 (Dense)                             (None, 6)           
BATCH SIZE 32, EPOCHS 2
Epoch 1/2
144992/159571 [==========================>...] - ETA: 3:02 - loss: 0.0555 - acc: 0.9805

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



159571/159571 [==============================] - 1987s 12ms/step - loss: 0.0423 - acc: 0.9839
Completed.


In [110]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv(f'{path}sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission_3_11feb.csv', index=False)

153164/153164 [==============================] - 18s 115us/step


In [ ]:
!ls